# **Interview task**
Create a jupyter notebook to train a segmentation model for damage detection based on the given training data.

In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import re
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

# Import Mask RCNN
from mrcnn.config import Config
from mrcnn import model as modellib, utils

/home/luca/interview/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/luca/interview/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/luca/interview/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/luca/interview/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
dataset = os.path.join(".", "dataset")
test_images = os.path.join(".", "test")
logs = weights_path = os.path.join(".", "logs")
weights_path = os.path.join(".", "mask_rcnn_coco.h5")

### **Configurations**

In [3]:
class CustomConfig(Config):
    """
    Configuration to train on the given dataset that overrides some of the values
    """
    NAME = "damage"
    IMAGES_PER_GPU = 2 # GPU with 12GB memory can fit two images.
    NUM_CLASSES = 1 + 1  # Background + damage
    STEPS_PER_EPOCH = 10
    DETECTION_MIN_CONFIDENCE = 0.9 # Skip detections with < 90% confidence
    

class InferenceConfig(CustomConfig):
    """
    Configuration to evaluate the model on test images
    Set batch size to 1 (inference on one image at a time).
    """
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

### **Dataset**

In [4]:
class myDataset(utils.Dataset):

    def load_custom(self, dataset_dir, mode):
        
        # Add "damage" class
        self.add_class("damage", 1, "damage")

        # Train or validation dataset
        dataset_dir = os.path.join(dataset_dir, mode)

        # Load annotations (x and y coordinates of each region)
        annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations = list(annotations.values())

        # Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinates of polygon vertexes for each object instance
            polygons = [r['shape_attributes'] for r in a['regions'].values()]

            # retrieve the image size to convert polygons to masks.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "damage",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        
        # Convert polygons to a bitmap mask of shape [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        
        info = self.image_info[image_id]
        if info["source"] == "damage":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

### **Train**

In [8]:
def train(model):

    # Training dataset.
    dataset_train = myDataset()
    dataset_train.load_custom(dataset, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = myDataset()
    dataset_val.load_custom(dataset, "val")
    dataset_val.prepare()

    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers='heads')

### **Color masks functions**

In [6]:
def color_splash(image, mask):
    """
    Apply color splash effect.
    image: RGB image [height, width, 3]
    mask: instance segmentation mask [height, width, instance count]

    Returns result image.
    """
    # Make a grayscale copy of the image.
    gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
    # We're treating all instances as one, so collapse the mask into one layer
    mask = (np.sum(mask, -1, keepdims=True) >= 1)
    # Copy color pixels from the original color image where mask is set
    if mask.shape[0] > 0:
        splash = np.where(mask, image, gray).astype(np.uint8)
    else:
        splash = gray
    return splash


def detect_and_color_splash(model, image_path=None):
    """
    Run model detection and generate the color splash effect
    """
    print("Running on {}".format(image_path))
    # Read image
    image = skimage.io.imread(image_path)
    # Detect objects
    r = model.detect([image], verbose=1)[0]
    # Color splash
    splash = color_splash(image, r['masks'])
    # Save output
    file_name = "detected_damage_{}.png".format(image_path.split("/")[-1].split(".")[0])
    skimage.io.imsave(file_name, splash)

    print("Saved", file_name)

### **Training model**

In [ ]:
# Training configuration
config = CustomConfig()
# config.display()

# Define model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=logs)

# Loading pre-trained weights excluding the last layers because they require a matching number of classes
print("Loading weights from", weights_path)
model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

# Train model
train(model)

### **Damage detection on test images**

In [11]:
# Inference configuration   
config = InferenceConfig()
# config.display()

# Create model in inference mode
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=logs)

# Loading pre-trained weights excluding the last layers because they require a matching number of classes
print("Loading weights from", weights_path)
model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

# Evaluate model on each image of the test folder
for filename in os.listdir(test_images):
    
    image = os.path.join(test_images, filename)
    detect_and_color_splash(model, image_path=image)

Loading weights from ./mask_rcnn_coco.h5
Running on ./test/test1.jpg
Processing 1 images
image                    shape: (1415, 2119, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 2119.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  detected_damage_20220311T185439.png
Running on ./test/test2.jpg
Processing 1 images
image                    shape: (360, 480, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Saved to  detected_damage_20220311T185450.png
